In [1]:
import jieba
import pandas as pd
import time
import re

In [2]:
content = pd.read_csv('../train/sqlResult_1558435.csv',encoding='gb18030')

In [3]:
content = content.fillna('')

In [4]:
all_content =' '.join(content['content'].tolist())

In [5]:
len(all_content)

38818836

In [10]:
def cut(string): return list(jieba.cut(string))

In [14]:
def token(string):
    return re.findall(r'[\w\d]+',string)

In [15]:
all_content = ''.join(token(all_content))

In [16]:
all_cut_words = cut(all_content)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/_1/8g6kq5zs3h97jlvhdzj6dldc0000gp/T/jieba.cache
Loading model cost 0.804 seconds.
Prefix dict has been built succesfully.


In [19]:
with open('../news_corpus.txt', 'w', encoding='utf-8') as f:
    f.write(' '.join(all_cut_words))

In [22]:
with open('../train_wiki_news_corpus.txt', 'a') as output_f:
    output_f.write(' '.join(all_cut_words))

In [102]:
def cut_sent(para):
    para = re.sub('[\r]',r'',para)
    para = re.sub('([。！？\?])([^”])',r"\1\n\2",para) # 单字符断句符
    para = re.sub('(\.{6})([^”])',r"\1\n\2",para) # 英文省略号
    para = re.sub('(\…{2})([^” ) ）])',r"\1\n\2",para) # 中文省略号
    #para = re.sub('(”)','”\n',para)   # 把分句符\n放到双引号后，注意前面的几句都小心保留了双引号
    para = para.rstrip()       # 段尾如果有多余的\n就去掉它
    #很多规则中会考虑分号;，但是这里我把它忽略不计，破折号、英文双引号等同样忽略，需要的再做些简单调整即可。
    return para.split("\n")

In [103]:
new_content = '\n'.join(content.content)

In [128]:
#切句
with open('../train/news_cut_sentence.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(x for x in cut_sent(new_content) if x))

In [6]:
sen=[]
with open('../train/news_cut_sentence.txt', 'r', encoding='utf-8') as f:
    sen=f.read().split('\n')

In [11]:
from gensim.models import Word2Vec

In [12]:
from gensim.models.word2vec import LineSentence

In [4]:
s = time.time()
model = Word2Vec(LineSentence('../train_wiki_news_corpus.txt'), min_count=5, size=40,workers=8)
e = time.time()
print(e-s)

1640.1000580787659


In [5]:
model.save('../word2vecModel/wiki_news_corpus_model_5_40')

In [9]:
model.wv.most_similar('数学',topn=30)

[('微积分', 0.9190504550933838),
 ('数学分析', 0.9046077132225037),
 ('数论', 0.8968438506126404),
 ('逻辑学', 0.8888925313949585),
 ('数理逻辑', 0.8845503926277161),
 ('概率论', 0.8833284974098206),
 ('拓扑学', 0.8758421540260315),
 ('纯数学', 0.8646337985992432),
 ('理论物理学', 0.8636034727096558),
 ('计算机科学', 0.859412670135498),
 ('高等数学', 0.8561244606971741),
 ('三角学', 0.8522976040840149),
 ('理论物理', 0.8500341773033142),
 ('群论', 0.8463723659515381),
 ('量子场论', 0.8457542657852173),
 ('电磁学', 0.8452684879302979),
 ('算术', 0.8416410684585571),
 ('量子力学', 0.8410530090332031),
 ('物理学', 0.8398241996765137),
 ('语言学', 0.8377435207366943),
 ('微积分学', 0.8360921740531921),
 ('统计学', 0.8349641561508179),
 ('数学知识', 0.8318045139312744),
 ('解析几何', 0.8316671252250671),
 ('统计力学', 0.8308815956115723),
 ('线性代数', 0.8291342258453369),
 ('代数学', 0.8272193670272827),
 ('哲学', 0.8266081213951111),
 ('物理', 0.8262129426002502),
 ('力学', 0.8240675926208496)]

In [3]:
model111 = Word2Vec.load('../word2vecModel/wiki_news_corpus_model_5_40')

In [5]:
model111.wv.most_similar('中国')

[('大陆', 0.7929396629333496),
 ('我国', 0.788699746131897),
 ('内地', 0.7753056287765503),
 ('韩国', 0.7735400795936584),
 ('中华人民共和国', 0.7675270438194275),
 ('全国', 0.7519695162773132),
 ('东亚', 0.7358080744743347),
 ('欧美', 0.7326491475105286),
 ('国内', 0.7224422097206116),
 ('各国', 0.7195731997489929)]

## get related words (说)

In [32]:
from collections import defaultdict

In [33]:
def get_related_words(initial_words, model):
    max_size = 300
    seen = defaultdict(int)
    unseen = [initial_words]
    
    while unseen and len(seen) < max_size:
        if len(seen) % 100 == 0:
            print('seen length: {}'.format(len(seen)))
        node = unseen.pop(0)

        new_expanding = [w for w, s in model.most_similar(node, topn=30)]
        
        unseen += new_expanding
        
        seen[node] += 1  
        # if we need more sophsiticated, we need change the value as the function(layer, similarity)
    seen = sort_by_count_and_similarity(initial_words,seen,model)
    return seen

In [34]:
def sort_by_count(nodes,model):
    return sorted(nodes.items(), key=lambda x: x[1], reverse=True)

In [35]:
def sort_by_count_and_similarity(initial_words,nodes,model):
    return sorted(nodes.items(), key=lambda x: x[1]*0.1+model.wv.similarity(initial_words,x[0])*0.9, reverse=True)

In [8]:
related_words = get_related_words('说', model111)

seen length: 0


/Users/Gago/Workplace/Software/Anaconda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # This is added back by InteractiveShellApp.init_path()


seen length: 100
seen length: 100
seen length: 200
seen length: 300
seen length: 400


In [15]:
model111.wv.most_similar('说',topn=50)

[('问道', 0.8809996247291565),
 ('问', 0.8770667314529419),
 ('回答', 0.867761492729187),
 ('说道', 0.8495763540267944),
 ('反问', 0.8487993478775024),
 ('却说', 0.8391307592391968),
 ('感叹', 0.8343255519866943),
 ('所说', 0.8281114101409912),
 ('告诉', 0.8260884881019592),
 ('答道', 0.8256169557571411),
 ('告诫', 0.8029030561447144),
 ('说出', 0.786902666091919),
 ('问起', 0.7853075265884399),
 ('要说', 0.7842749357223511),
 ('质问', 0.7834882736206055),
 ('打趣', 0.7785637378692627),
 ('指出', 0.7749508619308472),
 ('常说', 0.7744086980819702),
 ('直言', 0.7738279700279236),
 ('地说', 0.7734838724136353),
 ('想想', 0.7702670693397522),
 ('明白', 0.7682356834411621),
 ('知道', 0.7679475545883179),
 ('文说', 0.7657732963562012),
 ('他称', 0.7641598582267761),
 ('慨叹', 0.7631779909133911),
 ('认为', 0.761385440826416),
 ('骂', 0.7561197280883789),
 ('时说', 0.7559938430786133),
 ('答', 0.7556079030036926),
 ('问过', 0.7549947500228882),
 ('明说', 0.7532150149345398),
 ('深信', 0.7518712282180786),
 ('吹嘘', 0.7513055205345154),
 ('称', 0.75022155046

In [37]:
s = time.time()
model = Word2Vec(LineSentence('../train/train_wiki_news_corpus.txt'), min_count=1, size=100,workers=8)
e = time.time()
print(e-s)

2083.4742789268494


In [39]:
model.wv.most_similar('说',topn=50)

[('说道', 0.7962644100189209),
 ('指出', 0.7750760316848755),
 ('却说', 0.7660485506057739),
 ('反问', 0.7601529359817505),
 ('问', 0.7598448991775513),
 ('所说', 0.7582904696464539),
 ('问道', 0.7556657195091248),
 ('告诉', 0.7554758191108704),
 ('声称', 0.7288131713867188),
 ('回答', 0.7264024019241333),
 ('文说', 0.7223842144012451),
 ('所言', 0.7169530391693115),
 ('答道', 0.716424822807312),
 ('时说', 0.7014147043228149),
 ('告诫', 0.6953698396682739),
 ('直言', 0.6940948963165283),
 ('表示', 0.6939635276794434),
 ('明说', 0.691307008266449),
 ('写道', 0.6882731318473816),
 ('认为', 0.6880528926849365),
 ('答', 0.683323860168457),
 ('慨叹', 0.6830087900161743),
 ('坚称', 0.6771796345710754),
 ('感叹', 0.6769365072250366),
 ('觉得', 0.6764885187149048),
 ('相信', 0.6756546497344971),
 ('宣称', 0.673690915107727),
 ('提到', 0.6731791496276855),
 ('质问', 0.6722986698150635),
 ('坦言', 0.6684993505477905),
 ('地说', 0.6680620312690735),
 ('明言', 0.6658352017402649),
 ('中说', 0.665518045425415),
 ('辩称', 0.6651843786239624),
 ('说出', 0.66271883249

In [40]:
model.save('../word2vecModel/wiki_news_corpus_model_1_100')

In [13]:
model = Word2Vec.load('../word2vecModel/wiki_news_corpus_model_1_50')

In [41]:
related_words = get_related_words('说', model)

seen length: 0


/Users/Gago/Workplace/Software/Anaconda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # This is added back by InteractiveShellApp.init_path()


seen length: 100
seen length: 200
seen length: 200


In [43]:
related_words[:100]

[('说', 22),
 ('反问', 18),
 ('却说', 17),
 ('问道', 16),
 ('问', 15),
 ('说道', 14),
 ('答道', 14),
 ('慨叹', 12),
 ('回答', 11),
 ('告诫', 11),
 ('感叹', 10),
 ('质问', 10),
 ('告诉', 9),
 ('指出', 8),
 ('声称', 8),
 ('直言', 8),
 ('大骂', 8),
 ('曾言', 8),
 ('答', 7),
 ('相信', 7),
 ('宣称', 7),
 ('坦言', 7),
 ('写道', 6),
 ('辩称', 6),
 ('断言', 6),
 ('所说', 5),
 ('坦承', 6),
 ('骂', 6),
 ('文说', 5),
 ('叮嘱', 6),
 ('坚称', 5),
 ('提到', 5),
 ('他称', 5),
 ('确信', 5),
 ('还称', 5),
 ('怒斥', 5),
 ('脱口而出', 5),
 ('时说', 4),
 ('认为', 4),
 ('说出', 4),
 ('谈到', 4),
 ('吹嘘', 4),
 ('指称', 4),
 ('并不认为', 4),
 ('反驳', 4),
 ('所言', 3),
 ('信中称', 4),
 ('喊道', 4),
 ('请问', 4),
 ('戏言', 4),
 ('询问', 4),
 ('表示', 3),
 ('痛骂', 4),
 ('指责', 4),
 ('问起', 4),
 ('明言', 3),
 ('告戒', 4),
 ('看来', 3),
 ('原话', 3),
 ('一再强调', 3),
 ('责备', 3),
 ('大喊', 3),
 ('暗示', 3),
 ('说完', 3),
 ('自问', 4),
 ('自述', 3),
 ('解释', 3),
 ('自言', 3),
 ('直说', 3),
 ('强调', 3),
 ('明说', 2),
 ('追问', 3),
 ('责怪', 3),
 ('问过', 3),
 ('觉得', 2),
 ('透露', 3),
 ('所述', 3),
 ('地说', 2),
 ('中说', 2),
 ('否认', 3),
 ('表明', 3),
 ('称', 2),
 (

In [44]:
say_words = set()

In [45]:
for i in related_words[:100]:
    say_words.add(i[0])

In [46]:
with open('../train/say_words.txt','w+',encoding='utf-8') as outputf:
    outputf.write(' '.join(say_words))

## get word frequency

In [22]:
word_frequency = {}

In [24]:
with open('../train/train_wiki_news_corpus.txt', 'r') as output_f:
    sentences = output_f.read()

In [32]:
model.wv.vocab['中国'].count

434193

In [46]:
words_total = len(sentences.split())

In [47]:
words_total

257702499

In [73]:
#from tqdm import tqdm_notebook

In [ ]:
#for c in tqdm_notebook(model.wv.vocab, total=len(model.wv.vocab[w].count)):
#   word_frequency[w]=model.wv.vocab[w].count/words_total

In [52]:
for w in model.wv.vocab:
    word_frequency[w]=model.wv.vocab[w].count/words_total

In [2]:
import json

In [67]:
with open('../train/word_frequency.txt','w') as file:
    file.write(json.dumps(word_frequency))

In [3]:
with open('../train/word_frequency.txt','r') as file:
    sf=json.loads(file.read())